In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df=pd.read_csv('processed.csv')
print('Data shape:',df.shape)

In [ ]:
list(enumerate(df.columns))

1-hot encoding

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

Decision Tree

In [ ]:
acc_train=[]
acc_test=[]
param_grid={
    'criterion': ['gini', 'entropy', 'log_loss'],
    'ccp_alpha': [0.1, 0.01, 0.001],
    'max_depth': [None, 5, 6, 7, 8, 9, 10, 20, 40, 100, 200, 500],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': range(1,7)
}
for train_index, test_index in StratifiedKFold().split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    clf = DecisionTreeClassifier(random_state=0)
    grid=GridSearchCV(clf,param_grid,cv=5,n_jobs=-1)
    grid.fit(X_train, y_train)
    acc_train.append(accuracy_score(y_train,grid.predict(X_train)))
    acc_test.append(accuracy_score(y_test,grid.predict(X_test)))
print('Test acc:',np.mean(acc_test))
print('Train acc:',np.mean(acc_train))
plt.plot(range(1,6),acc_train)
plt.plot(range(1,6),acc_test)
plt.legend(['train','test'])